In [3]:
%matplotlib inline
from fns import *
from syntheticdata import synthetic_generation
# import ipyvolume

In [4]:
import plotly
plotly.tools.set_credentials_file(username='gpernelle', api_key='4KIdNJBBApIeebdKxyN9')
import plotly.plotly as py
import plotly.graph_objs as go


In [5]:
def dice(logits, labels):
        flat_logits = logits.flatten()
        flat_labels = labels.flatten()
        intersection = np.sum(flat_logits*flat_labels)
        union = np.sum(flat_logits) + np.sum(flat_labels)
        loss = 1 - 2 * intersection / union
        return loss

### Predict

In [24]:
def predict_case(caseNumber):
    case=str(caseNumber)
    image_name = '/home/gp1514/SSD/preprocessed_data/LabelMapsNEW2_1.00-1.00-1.00_split2/%s/case.nrrd'%case
    label_name = '/home/gp1514/SSD/preprocessed_data/LabelMapsNEW2_1.00-1.00-1.00/%s/needles.nrrd'%case

    data, options_ = nrrd.read(image_name)
    data = data.astype(np.float32)
    data1, options = nrrd.read(image_name.replace('_split2', ''))
    data1 = data1.astype(np.float32)

    label_data, options_ = nrrd.read(label_name)

    prediction = predict([data1,data], "./models/unet_trained_mix_dropout-05-clahe30_f32")
    return label_data, prediction, options


def save_case(caseNumber):
    case=str(caseNumber)
    image_name = '/home/gp1514/SSD/preprocessed_data/LabelMapsNEW2_1.00-1.00-1.00_split2/%s/case.nrrd'%case
    data1, options = nrrd.read(image_name.replace('_split2', ''))
    data1 = data1.astype(np.float32)
    nrrd.write('predictions/case_%s.nrrd'%str(caseNumber), data1, options=options)



In [18]:
def predict_and_save(caseNumber):
    label_data, prediction, options = predict_case(caseNumber)
    dice(prediction, label_data)
    islands = post_processing(prediction, min_area=int(50), max_residual=float(4))
    nrrd.write('predictions/%s.nrrd'%str(caseNumber), islands, options=options)

def predict_and_plot3d(caseNumber):
    label_data, prediction, options = predict_case(caseNumber)
    dice(prediction, label_data)
    islands = post_processing(prediction, min_area=int(50), max_residual=float(4))
    nrrd.write('predictions/%s.nrrd'%str(caseNumber), islands, options=options)
    x,y,z = np.where(label_data == 1)
    xs,ys,zs = np.where(islands != 0)

    trace1 = go.Scatter3d(
        x=x,
        y=y,
        z=z,
        mode='markers',
        marker=dict(
            size=3,
            line=dict(
                color='green',
                width=2
            ),
            opacity=0.3
        )
    )

    trace2 = go.Scatter3d(
        x=xs,
        y=ys,
        z=zs,
        mode='markers',
        marker=dict(
            color='red',
            size=3,
            symbol='circle',
            line=dict(
                color='rgb(204, 204, 204)',
                width=1
            ),
            opacity=0.3
        )
    )
    data = [trace1, trace2]
    layout = go.Layout(
        margin=dict(
            l=0,
            r=0,
            b=0,
            t=0
        )
    )

    fig = go.Figure(data=data, layout=layout)
    py.iplot(fig, filename='simple-3d-scatter')

In [8]:
validationCases = loadCases("preprocessing/validation.txt")
testingCases = loadCases("preprocessing/testing.txt")

In [9]:
validationCases

['021', '023', '037', '038', '070', '071', '072', '074', '075', '076', '077']

In [10]:
testingCases

['028', '051', '054']

In [15]:
for case in validationCases:
    predict_and_save(case)

--------------------------------------------------
Loading and Preparing the data
Original input shape (200, 200, 77)
Padded input shape: (288, 288, 165)
# of parts 5 5 3
number of tiles: 75 
data shape (75, 148, 148, 148)
Original input shape (200, 200, 77)
Padded input shape: (288, 288, 165)
# of parts 5 5 3
number of tiles: 75 
data shape (75, 148, 148, 148)


100%|██████████| 75/75 [00:00<00:00, 639375.61it/s]

--------------------------------------------------
Starting the segmenter.


imgs shape (75, 148, 148, 148, 2)
Feature shape:  (75, 148, 148, 148, 2)
INFO:tensorflow:Restoring parameters from ./models/unet_trained_mix_dropout-05-clahe30_f32/model 3.cpkt


100%|██████████| 75/75 [00:32<00:00,  2.61it/s]


...

100%|██████████| 96/96 [00:00<00:00, 958698.06it/s]

--------------------------------------------------
Starting the segmenter.


imgs shape (96, 148, 148, 148, 2)
Feature shape:  (96, 148, 148, 148, 2)
INFO:tensorflow:Restoring parameters from ./models/unet_trained_mix_dropout-05-clahe30_f32/model 4.cpkt


100%|██████████| 96/96 [00:37<00:00,  2.32it/s]


prediction shape: (96, 1, 60, 60, 60, 1)
Merging subvolumes
--------------------------------------------------
Padded input shape: (388, 283, 254)
# of parts 7 7 5
# of subvolumes merged:  96
(array([0, 1, 2, 3, 4], dtype=uint16), array([9702724,    6109,    1808,     303,      56]))
(300, 195, 166)
Number of regions: 46
1 0.900571086929 53
2 1.12641023453 77
3 0.943084434411 91
4 0.98832450191 66
5 1.09172542506 274
6 1.34124100697 172
7 1.85746295917 366
8 1.06043336577 129
9 1.02879602786 111
10 1.05943386723 344
11 1.09712349491 438
12 3.03748097232 857
13 0.958126135094 66
14 1.131542999 574
15 1.10704701568 522
16 0.923792173739 68
17 0.998394391793 110
18 1.07250946042 103
19 1.12413642399 61


In [16]:
for case in testingCases:
    predict_and_save(case)

--------------------------------------------------
Loading and Preparing the data
Original input shape (220, 220, 204)
Padded input shape: (308, 308, 292)
# of parts 6 6 5
number of tiles: 125 
data shape (125, 148, 148, 148)
Original input shape (220, 220, 204)
Padded input shape: (308, 308, 292)
# of parts 6 6 5
number of tiles: 125 
data shape (125, 148, 148, 148)


100%|██████████| 125/125 [00:00<00:00, 825650.39it/s]

--------------------------------------------------
Starting the segmenter.


imgs shape (125, 148, 148, 148, 2)
Feature shape:  (125, 148, 148, 148, 2)
INFO:tensorflow:Restoring parameters from ./models/unet_trained_mix_dropout-05-clahe30_f32/model 5.cpkt


100%|██████████| 125/125 [00:56<00:00,  2.50it/s]


...

100%|██████████| 245/245 [00:00<00:00, 1102579.91it/s]

--------------------------------------------------
Starting the segmenter.


imgs shape (245, 148, 148, 148, 2)
Feature shape:  (245, 148, 148, 148, 2)
INFO:tensorflow:Restoring parameters from ./models/unet_trained_mix_dropout-05-clahe30_f32/model 5.cpkt


100%|██████████| 245/245 [01:40<00:00,  2.59it/s]


prediction shape: (245, 1, 60, 60, 60, 1)
Merging subvolumes
--------------------------------------------------
Padded input shape: (407, 407, 308)
# of parts 7 7 6
# of subvolumes merged:  245
(array([0, 1, 2, 3], dtype=uint16), array([22377834,     9440,       36,      110]))
(319, 319, 220)
Number of regions: 35
1 2.75415777664 440
2 0.958465043059 74
3 1.05409727428 190
4 1.01509114766 132
5 2.16910435777 1096
6 1.12314136338 333
7 1.07016822832 134
8 1.07382002294 137
9 1.00608888793 95
10 1.10937917918 582
11 1.06624199159 119
12 1.09510772297 619
13 0.964160450633 64
14 1.12418258231 683
15 1.10125371609 736
16 1.15520018706 775
17 1.02833033435 131
18 1.1199639177 574
19 1.16176050466 624
20 1.07907222484 219


In [25]:
for case in validationCases:
    save_case(case)

In [27]:
predict_and_plot3d('071')

--------------------------------------------------
Loading and Preparing the data
Original input shape (300, 195, 230)
Padded input shape: (388, 283, 318)
# of parts 7 7 6
number of tiles: 120 
data shape (120, 148, 148, 148)
Original input shape (300, 195, 230)
Padded input shape: (388, 283, 318)
# of parts 7 7 6
number of tiles: 120 
data shape (120, 148, 148, 148)


100%|██████████| 120/120 [00:00<00:00, 1157049.38it/s]

--------------------------------------------------
Starting the segmenter.


imgs shape (120, 148, 148, 148, 2)
Feature shape:  (120, 148, 148, 148, 2)
INFO:tensorflow:Restoring parameters from ./models/unet_trained_mix_dropout-05-clahe30_f32/model 36.cpkt


100%|██████████| 120/120 [00:58<00:00,  2.15it/s]


prediction shape: (120, 1, 60, 60, 60, 1)
Merging subvolumes
--------------------------------------------------
Padded input shape: (388, 283, 318)
# of parts 7 7 6
# of subvolumes merged:  120
(array([0, 1, 2, 3, 4, 5, 6], dtype=uint16), array([13428109,    20812,     5193,      521,      206,       96,       63]))
(300, 195, 230)
Number of regions: 92
1 0.98763646574 51
2 1.02812529399 105
3 1.04107969125 72
4 1.0012851112 121
5 1.05151365476 109
6 1.12319180549 1009
7 1.10252404356 853
8 1.09312485648 217
9 0.9555976602 124
10 2.89284458249 1270
11 1.17048515054 933
12 1.00609381915 85
13 1.08516779962 181
14 1.0064805911 101
15 1.03475136384 132
16 1.2745450089 239
17 1.13800702657 369
18 1.09530288982 285
19 1.32960747025 306
20 1.1024755718 655
21 1.09802860691 278
22 3.37097576361 2096
23 1.14908229273 507
24 0.994609393359 130
25 1.08047846948 276
26 1.1048067992 746
27 0.924951826031 54
28 0.961881228976 73
29 1.07492767629 88
30 2.90763647956 481
31 1.04583439235 226
32 1.182